# Some general ideas related to game theory

## Poker Problem 

Kuhn Poker: 1,2,3 -> 

Two pleyer
$$a_{}$$

## Adversarial Generation of Short Prompts for Improving Feedback Quality

**Motivation**

User feedback (reviews, evaluations, surveys) is often affected by cognitive biases, emotional tone, and low informational density. Respondents tend to provide vague, emotionally charged, or socially biased answers, which reduces the usefulness of feedback for decision-making and analysis. This problem is especially pronounced in educational evaluations, product reviews, and employee assessments.


How can short textual prompts be adaptively generated to maximize the information content and reduce bias in customer feedback, under adversarial evaluation?

feedback quality metrics
- Information content I(r) (number of distinct product aspects mentioned, or entropy of aspect distribution, or coverage of predefined aspect set)
- Bias metric B(r) (sentiment extremeness (absolute sentiment score), demographic skew (if groups exist), lexical subjectivity score.)
- Utility U(r) (Human usefulness. Collect later via human ratings 1–5. At the beginning, can be ignored)

GAME:
Generator (G) produces a short prompt p, Evaluator (E) evaluates the resulting feedback r.
G: finite set of prompt types (templates); E: fixed scoring function (at first)

Payoff: $u_{G}​(p)=\alpha I(r)−\beta B(r)$
Evaluator payoff: $uE​(p)=−uG​(p)$ -- this is for now a zero-sum game

The game is between a Prompt Generator and an Evaluator of feedback quality, where the generator tries to elicit informative, low-bias feedback, and the evaluator penalizes bias and low information.

BASELINE PIPELINE:
- Create a small prompt set - pure strategies
- Collect responses (may simulate using LLM)
- NLP analysis: extract aspects (rule-based or simple classifier), compute sentiment score.

payoff matrix

Now: eliminate dominated prompts, compute mixed strategies, identify equilibrium distributions.

FURTHER:
adversarial learning
- Parameterize the prompt - Instead of fixed templates, represent prompt as (length,specificity,number of constraints,presence of examples) -> vector $\theta$
- Train generator, using RL (action = generate prompt parameters, reward = $u_{G}$, environment = evaluator + NLP pipeline.)
- 

comparison of prompts before and after optimization, statistical improvement in I and reduction in B, robustness analysis across datasets.

## Adversarial Generation of Short Prompts for Improving Feedback Quality

customer feedback in textual form

$$R=\{all possible feedback texts\}$$

$$P=\{all possible short prompts\}$$

Given a prompt $p\in P$, the user (environment) generates a feedback text:

$$r∼P(⋅∣p),r\in R$$

The distribution is unknown and stochastic

In [7]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
SECRET_KEY = os.getenv("OPEN_API_KEY")

PROMPTS = [
    "What did you like about the product?",
    "What problems did you face while using the product?",
    "Please describe your experience with the product.",
    "What should we improve in the product?",
    "How satisfied are you with the product and why?"
]

client = OpenAI(api_key=SECRET_KEY)

def generate_feedback(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a customer giving honest feedback."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.9,
        max_tokens=120
    )
    return response.choices[0].message.content



Let $A=\{ a_{1}, ...,a_{k} \}$ be a predefined set of product aspects.

Define an aspect extraction function: $\phi :R\rightarrow 2^{A}$

Information content: $I(r)=∣\phi (r)∣$

Lexical divercity:

$$I(r) = \frac{unique tokens}{total tokens}$$

DEFINE BETTER FUNCTION LATER!

Bias metric:

$$B(r)=|s(r)|$$

Where 
$s(r)\in [−1,1] $ - sentiment polarity score

In [8]:
import re
from textblob import TextBlob


def information_metric(text):
    tokens = re.findall(r"\w+", text.lower())
    if len(tokens) == 0:
        return 0.0
    return len(set(tokens)) / len(tokens)

def bias_metric(text):
    sentiment = TextBlob(text).sentiment.polarity
    return abs(sentiment)

Empirical expectation (Monte Carlo)

Since feedback is stochastic, we work with expectations.

$$E[B∣p]=E_{r∼P(⋅∣p)}​[B(r)]$$
$$E[I∣p]=E_{r∼P(⋅∣p)}​[I(r)]$$


In [ ]:
import numpy as np

def estimate_expectations(prompt, n_samples=10):
    infos, biases = [], []

    for i in range(n_samples):
        r = generate_feedback(prompt)
        infos.append(information_metric(r))
        biases.append(bias_metric(r))

    return np.mean(infos), np.mean(biases)

**Payoff function (parametrized)**

Player G (Generator) chooses prompt p

Player E (Evaluator) enforces quality constraints

Strategy spaces

Generator strategies:

$p\in P$

Evaluator strategies:

$$\lambda = (\alpha, \beta)\in \mathbb{R}_{+}^{2}$$
	​
Evaluator controls the penalty weights.

Generator payoff:

$$u_G​(p,λ)=αE[I∣p]−βE[B∣p]$$

Evaluator payoff:

$$u_E​(p,λ)=−u_G​(p,λ)$$

**Static game with finite strategies:**

Finite prompts: $P={p_1​,…,p_n​}$

Define payoff: $U_i​=αE[I∣p_i​]−βE[B∣p_i​]$

$p_{i}$ is dominated by $p_{j}$ if $Uj_​≥U_i​$. Dominated prompts can be eliminated.

We seek a minimax equillibrium $\max_{\pi} \min_{\lambda} U_{G}(\pi, \lambda)$


THINK ABOUT LATER: Dynamic learning formulation, 
Let prompts be generated by parameters: $p=g(\theta),\theta \in \mathbb{R}^{d}$

Constraints on parameters: $\alpha+\beta=1$, $u_{E}​=−u_{G}​−c(\beta)$ ($c(\beta)=\gamma \beta^{2}$)

In [ ]:
def payoff(prompt, alpha=1.0, beta=1.0, n_samples=10):
    I_hat, B_hat = estimate_expectations(prompt, n_samples)
    return alpha * I_hat - beta * B_hat

Playing the game (best response)

The generator chooses the best prompt given evaluator parameters.

THINK ABOUT LATER: Transformer as Prompt Generator

Prompt as a function of parameters, instead of choosing a prompt directly, define: $p_{\theta} =G_{\theta}​(z)$

In [11]:
def best_prompt(prompts, alpha, beta):
    scores = {}
    for p in prompts:
        scores[p] = payoff(p, alpha, beta)
    return max(scores, key=scores.get), scores

best, all_scores = best_prompt(PROMPTS, alpha=1.0, beta=0.7)

print("Best prompt:", best)
for p, s in all_scores.items():
    print(f"{p[:40]}... -> {s:.4f}")

BETAS = [0.2, 0.5, 1.0, 2.0]

for beta in BETAS:
    best, _ = best_prompt(PROMPTS, alpha=1.0, beta=beta)
    print(f"beta={beta:.1f} -> best prompt: {best}")

Best prompt: What problems did you face while using the product?
What did you like about the product?... -> 0.5445
What problems did you face while using t... -> 0.6946
Please describe your experience with the... -> 0.6130
What should we improve in the product?... -> 0.6290
How satisfied are you with the product a... -> 0.5694
beta=0.2 -> best prompt: What problems did you face while using the product?
beta=0.5 -> best prompt: What problems did you face while using the product?
beta=1.0 -> best prompt: What problems did you face while using the product?
beta=2.0 -> best prompt: What problems did you face while using the product?
